In [1]:
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import pickle

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras

import numpy as np
import os
import time
import json
import pandas as pd
import random

from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Embedding

In [3]:
path = "drive/MyDrive/nlp-project/"
model_path = path + 'models/10k_new/'
data_path = path + 'data/reddit_jokes.json'

In [4]:
from tensorflow import keras
model = keras.models.load_model(model_path + 'model-char-best-accuracy.hdf5')

In [5]:
tokenizer = pickle.load(open(model_path + 'tokenizer-char.pkl', "rb"))

In [20]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	while True:
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = tf.keras.preprocessing.sequence.pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		choices = model.predict(encoded)
		yhat = np.random.choice(len(choices[0]), p=choices[0])
		#print(choices[0][yhat-2:yhat+2])
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += '' + out_word
		if out_word == '❌':
			result.append('.')
			break
		result.append(out_word)
	return ''.join(result)

In [26]:
generate_seq(model, tokenizer, 100, "What do you call an")

'a.'

In [21]:
def clean_df(df):
    # Remove content after edit
    df["title"] = df["title"].str.replace(r'edit:.*', '')
    df["body"] = df["body"].str.replace(r'edit:.*', '')
    
    # Creating "joke" column
    df["joke"] = np.where(df["title"].str[:10] != df["body"].str[:10], df["title"] + " " + df["body"], df["body"])
    
    return df

In [22]:
def read(json_filename):
    df = pd.read_json(path_or_buf=json_filename,orient='records',compression="infer")
    print("All jokes len", len(df))
    df = clean_df(df.iloc[:number_of_jokes])
    print("Loaded", number_of_jokes)
    
    return df

In [23]:
number_of_jokes = 10000
jokes_df = read(data_path)

All jokes len 194553
Loaded 10000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [27]:
jokes_list = jokes_df['joke'].to_numpy()
n = 2
for joke in jokes_list[50:100]:
  joke_split = joke.split(" ")
  if len(joke_split) > 50:
    continue
  pepe = " ".join(joke_split[:-n])
  generated = generate_seq(model, tokenizer, 20, pepe)
  if generated.strip() == '.' or len(generated.split(" ")) > 50:
    continue
  print("**Original joke:**")
  print(joke)
  print("**Generated:**")
  print(pepe, "::", generated)
  print("\n")

**Original joke:**
I found a place where the recycling rate is 100% This post.
**Generated:**
I found a place where the recycling rate is 100% ::  when they excuse hind all are to dys goes leds and joe."

**then

the ban on now thought it and thouldnow"

the gass him. the cinst and just bartender bottle but"

"discover, tryace door is in it kept with a voice and many said, he atpoases?


ethold a 20 sralles this will enough..


**Original joke:**
Roses are gray. Violets are gray. I'm a dog.

Happy Valentine's Day!
**Generated:**
Roses are gray. Violets are gray. I'm a dog.

Happy ::  "would ben hat end," says another east eacinits armone with the limp and left and they do you do- fout cack arms, with off how to pointed about much interized now i fetrlewer!"

when a bibles barge and bunles, satan pass vod..


**Original joke:**
I hear the Mexicans are planning an attack on America Don't worry, we have our Trump card.
**Generated:**
I hear the Mexicans are planning an attack on America D